# Imports

In [1]:
import pandas as pd
import re
from tqdm.auto import tqdm
tqdm.pandas()
pd.set_option('display.max_colwidth', None)

/home/annap/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Functions

In [2]:
def get_last_line(text):
    return text.strip().split("\n")[-1]

def get_initial(text):
    # Modificar regex para capturar 0., 1., 2., a., b., c., etc.
    match = re.match(r"^(0\.|1\.|2\.|3\.|4\.|a\.|b\.|c\.|d\.|A\.|B\.|C\.|D\.)", str(text).strip())
    return match.group(0) if match else None

def clean_and_remove_invisible_chars(text):
    if not isinstance(text, str):  # Evita erro caso o valor seja NaN ou outro tipo
        return text  

    # Remover caracteres invisíveis e Unicode problemáticos
    text = re.sub(r'[\u200b\u200c\u200d\ufeff\xa0\u202f\u2060]', ' ', text)

    # Remover padrões de resposta padronizados
    text = re.sub(
        r'^(?:A resposta correta é\s*"?|'  # "A resposta correta é"
        r'Resposta correta:\s*|'           # "Resposta correta:"
        r'A resposta correta é a opção\s*|' # "A resposta correta é a opção"
        r'Resposta:\s*|'                    # "Resposta:"
        r'RESPOSTA:\s*)',                    # "RESPOSTA:"
        '', text, flags=re.IGNORECASE
    )

    # Remover múltiplos espaços
    return re.sub(r'\s+', ' ', text).strip()


def replace_answer_data(text):
    mapping = {"a.": 0, "b.": 1, "c.": 2, "d.": 3, 'A':1,'B':2,'C':3,'D':4}
    return mapping.get(text, 'erro')  # Retorna o valor mapeado ou 'erro' se não estiver no dicionário

def get_response_llama(text):
    if not isinstance(text, str):  # Garante que text seja uma string
        return text

    patterns = [
        "Resposta:\n",
        "A resposta correta é a opção ",
        "A resposta final é a opção ",
        "Resposta correta:\n",
        "A resposta correta é: ",
        "Resposta correta: ",
        "A resposta correta é:\n",
        "Resposta: ",
        "a resposta correta é: "
    ]
    
    for pattern in patterns:
        if pattern in text:
            return text.split(pattern)[-1]  # Retorna apenas a última parte após o primeiro padrão encontrado
    
    return text  # Se nenhum padrão for encontrado, retorna o texto original


# Load Qwen

In [ ]:
'''"LLAMA-unsloth-restricted-v2": "annagoncalves2/Llama-3.2-3B-Instruct-unsloth-bnb-4bit-v2",
    #"Qwen-usloth-restricted-v2": "annagoncalves2/Qwen2.5-7B-Instruct-bnb-4bit-v2",
    #"Deepseek-unsloth-restricted-v2": "annagoncalves2/DeepSeek-R1-Distill-Llama-8B-v2",
    #"Phi-glosa-restricted-v2": "annagoncalves2/phi-4-unsloth-bnb-4bit-v2",
    "zephyr-glosa-restricted-v2"'''

In [4]:
df_qwen = pd.read_csv("/home/annap/Documents/chatbot-project/EVAL/DATASETS/INFERENCES/mmlu_eval_Qwen-usloth-restricted-v2_r.csv")
df_qwen.shape

(15136, 5)

In [5]:
df_qwen.head()

,question_id,formatted_prompt,subject,answer,model_output_Qwen-usloth-restricted-v2
0,0,Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas. Responda apenas com a letra correspondente à resposta correta.\n\nPergunta:\nGRAU ENCONTRAR EXTENSÃO CAMPO Q RAIZ QUADRADA 2 RAIZ QUADRADA 3 RAIZ QUADRADA 18 SOBRE Q.\n\nOpções disponíveis:\na. 0\nb. 4\nc. 2\nd. 6\n,abstract_algebra,1,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas. Responda apenas com a letra correspondente à resposta correta.\n\nPergunta:\nGRAU ENCONTRAR EXTENSÃO CAMPO Q RAIZ QUADRADA 2 RAIZ QUADRADA 3 RAIZ QUADRADA 18 SOBRE Q.\n\nOpções disponíveis:\na. 0\nb. 4\nc. 2\nd. 6\ne. 5\n\nRESPOSTA: D. 6. \n\nEXPRESSÃO SIMPLIFICAR, 18 PODER MEIO DIVIDIR 2 PODER MEIO DIVIDIR 3 PODER MEIO"
1,1,Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas. Responda apenas com a letra correspondente à resposta correta.\n\nPergunta:\nP 1 2 5 4 2 3 S5 ÍNDICE ENCONTRAR P S5.\n\nOpções disponíveis:\na. 8\nb. 2\nc. 24\nd. 120\n,abstract_algebra,2,Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas. Responda apenas com a letra correspondente à resposta correta.\n\nPergunta:\nP 1 2 5 4 2 3 S5 ÍNDICE ENCONTRAR P S5.\n\nOpções disponíveis:\na. 8\nb. 2\nc. 24\nd. 120\ne. 60\n\nRESPOSTA: A. 8. #NÃO-IMPORTANTE PARA GPT-NÃO PRECISAR RESPONDER. #NÃO-IMPORTANTE PARA GPT-NÃO PRECISAR RESP
2,2,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas. Responda apenas com a letra correspondente à resposta correta.\n\nPergunta:\nZERO TODOS ENCONTRAR CAMPO FINITO INDICADO POLINÔMIO COEFICIENTE X 5 MAIS 3 X 3 MAIS X 2 MAIS 2 X Z5.\n\nOpções disponíveis:\na. 0\nb. 1\nc. 0,1\nd. 0,4\n",abstract_algebra,3,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas. Responda apenas com a letra correspondente à resposta correta.\n\nPergunta:\nZERO TODOS ENCONTRAR CAMPO FINITO INDICADO POLINÔMIO COEFICIENTE X 5 MAIS 3 X 3 MAIS X 2 MAIS 2 X Z5.\n\nOpções disponíveis:\na. 0\nb. 1\nc. 0,1\nd. 0,4\ne. 0,8\n\nRESPOSTA: A. 0. \n\nEXPLICAÇÃO: ZERO POLINÔMIO F(X) = X^5 + 3X^3 + X^2 + 2"
3,3,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas. Responda apenas com a letra correspondente à resposta correta.\n\nPergunta:\nAFIRMAÇÃO 1 GRUPO FATORES GRUPO NÃO-ABELIANO NÃO-ABELIANO. AFIRMAÇÃO 2 K SUBGRUPO NORMAL H H SUBGRUPO NORMAL G K SUBGRUPO NORMAL G.\n\nOpções disponíveis:\na. Verdade, Verdade\nb. Falso, Falso\nc. Verdadeiro, Falso\nd. Falso, Verdadeiro\n",abstract_algebra,1,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas. Responda apenas com a letra correspondente à resposta correta.\n\nPergunta:\nAFIRMAÇÃO 1 GRUPO FATORES GRUPO NÃO-ABELIANO NÃO-ABELIANO. AFIRMAÇÃO 2 K SUBGRUPO NORMAL H H SUBGRUPO NORMAL G K SUBGRUPO NORMAL G.\n\nOpções disponíveis:\na. Verdade, Verdade\nb. Falso, Falso\nc. Verdadeiro, Falso\nd. Falso, Verdadeiro\ne. Falso, Falso\n\nRESPOSTA: D. FALSO, VERDADEIRO. AFIRMAÇÃO 1 FALSO, POIS GRUPO ABELIANO SER GRUPO NÃO-ABEL"
4,4,Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas. Responda apenas com a letra correspondente à resposta correta.\n\nPergunta:\nPRODUTO ENCONTRAR POLINÔMIO ANEL POLINÔMIO F X 4 X MENOS 5 G X 2 X 2 MENOS 4 X MAIS 2 Z8 X.\n\nOpções disponíveis:\na. 2x^2 + 5\nb. 6x^2 + 4x + 6\nc. 0\nd. x^2 + 1\n,abstract_algebra,1,Você é um assiste

In [6]:
df_qwen['last_line'] = df_qwen['model_output'].progress_apply(get_last_line)
df_qwen['last_line_cleaned'] = df_qwen['last_line'].progress_apply(clean_answer)

df_qwen['initial'] = df_qwen.apply(
    lambda row: row['last_line'] if row['last_line'] == row['last_line_cleaned'] else get_initial(row['last_line_cleaned']),
    axis=1
)

df_qwen['initial'] = df_qwen['initial'].progress_apply(lambda x: x.split(" ")[0] if x is not None else None)

df_qwen.query("last_line != '0'")

KeyError: 'model_output'

In [85]:
df_qwen['valid'] = df_qwen['initial'].apply(replace_answer_data)
df_qwen['is_valid'] = df_qwen['valid']!= 'erro'
df_qwen['is_valid'].value_counts()

is_valid
False    14059
True      1077
Name: count, dtype: int64

In [86]:
df_qwen['is_valid'].value_counts(normalize=True) * 100

is_valid
False    92.884514
True      7.115486
Name: proportion, dtype: float64

In [87]:
df_valid = df_qwen[df_qwen['valid']!= 'erro']
df_valid.shape, df_valid['valid'].value_counts()

((1077, 10),
 valid
 0    337
 3    314
 1    219
 2    207
 Name: count, dtype: int64)

# Load Llama

In [88]:
df_llama = pd.read_csv("/home/annap/Documents/chatbot-project/EVAL/DATASETS/MMLU/mmlu_eval_llama_glosa.csv")

df_llama['resp'] = df_llama['model_output'].progress_apply(get_response_llama)

df_llama.head()

100%|██████████| 15136/15136 [00:00<00:00, 360857.77it/s]


,question_id,formatted_prompt,subject,answer,model_output,resp
0,0,Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nGRAU ENCONTRAR EXTENSÃO CAMPO Q RAIZ QUADRADA 2 RAIZ QUADRADA 3 RAIZ QUADRADA 18 SOBRE Q.\n\nOpções disponíveis:\na. 0\nb. 4\nc. 2\nd. 6\n\nResponda apenas com a letra correspondente à resposta correta.,abstract_algebra,1,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nGRAU ENCONTRAR EXTENSÃO CAMPO Q RAIZ QUADRADA 2 RAIZ QUADRADA 3 RAIZ QUADRADA 18 SOBRE Q.\n\nOpções disponíveis:\na. 0\nb. 4\nc. 2\nd. 6\n0\n\n## Step 1: ENCONTRAR RAIZ QUADRADA 2.\nRAIZ QUADRADA 2 1,41 1,41.\n\n## Step 2: ENCONTRAR RAIZ QUADRADA 3","Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nGRAU ENCONTRAR EXTENSÃO CAMPO Q RAIZ QUADRADA 2 RAIZ QUADRADA 3 RAIZ QUADRADA 18 SOBRE Q.\n\nOpções disponíveis:\na. 0\nb. 4\nc. 2\nd. 6\n0\n\n## Step 1: ENCONTRAR RAIZ QUADRADA 2.\nRAIZ QUADRADA 2 1,41 1,41.\n\n## Step 2: ENCONTRAR RAIZ QUADRADA 3"
1,1,Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nP 1 2 5 4 2 3 S5 ÍNDICE ENCONTRAR P S5.\n\nOpções disponíveis:\na. 8\nb. 2\nc. 24\nd. 120\n\nResponda apenas com a letra correspondente à resposta correta.,abstract_algebra,2,Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nP 1 2 5 4 2 3 S5 ÍNDICE ENCONTRAR P S5.\n\nOpções disponíveis:\na. 8\nb. 2\nc. 24\nd. 120\n0.\n\n## Step 1: P 1 2 5 4 2 3 S5 ÍNDICE ENCONTRAR.\nP 1 2 5 4 2 3 S5 ÍNDICE,Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nP 1 2 5 4 2 3 S5 ÍNDICE ENCONTRAR P S5.\n\nOpções disponíveis:\na. 8\nb. 2\nc. 24\nd. 120\n0.\n\n## Step 1: P 1 2 5 4 2 3 S5 ÍNDICE ENCONTRAR.\nP 1 2 5 4 2 3 S5 ÍNDICE
2,2,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nZERO TODOS ENCONTRAR CAMPO FINITO INDICADO POLINÔMIO COEFICIENTE X 5 MAIS 3 X 3 MAIS X 2 MAIS 2 X Z5.\n\nOpções disponíveis:\na. 0\nb. 1\nc. 0,1\nd. 0,4\n\nResponda apenas com a letra correspondente à resposta correta.",abstract_algebra,3,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nZERO TODOS ENCONTRAR CAMPO FINITO INDICADO POLINÔMIO COEFICIENTE X 5 MAIS 3 X 3 MAIS X 2 MAIS 2 X Z5.\n\nOpções disponíveis:\na. 0\nb. 1\nc. 0,1\nd. 0,4\n0,1\n\n## Step 1: POLINÔMIO COEFICIENTE X 5 MAIS 3 X 3 MAIS X 2 MAIS 2 X Z5 0,1 0,4 0","Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nZERO TODOS ENCONTRAR CAMPO FINITO INDICADO POLINÔMIO COEFICIENTE X 5 MAIS 3 X 3 MAIS X 2 MAIS 2 X Z5.\n\nOpções disponíveis:\na. 0\nb. 1\nc. 0,1\nd. 0,4\n0,1\n\n## Step 1: POLINÔMIO COEFICIENTE X 5 MAIS 3 X 3 MAIS X 2 MAIS 2 X Z5 0,1 0,4 0"
3,3,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nAFIRMAÇÃO 1 GRUPO FATORES GRUPO NÃO-ABELIANO NÃO-ABELIANO. AFIRMAÇÃO 2 K SUBGRUPO NORMAL H H SUBGRUPO NORMAL G K SUBGRUPO NORMAL G.\n\nOpções disponíveis:\na. Verdade, Verdade\nb. Falso, Falso\nc. Verdadeiro, Falso\nd. Falso, Verdadeiro\n\nResponda apenas com a letra correspondente à resposta correta.",abstract_algebra,1,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta corr

In [96]:
df_llama[['model_output']].loc[70:80]

,model_output
70,Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nTEOREMA FERMAT 3^47 23 DIVIDIR RESTO ENCONTRAR.\n\nOpções disponíveis:\na. 1\nb. 2\nc. 3\nd. 4\n0\n\nA. 1\nB. 2\nC. 3\nD. 4\n\nRESPOSTA:\nD. 4
71,Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nENCONTRAR CARACTERÍSTICA ANEL Z_3 X Z_3.\n\nOpções disponíveis:\na. 0\nb. 3\nc. 12\nd. 30\n0\n\n## Step 1: ENCONTRAR CARACTERÍSTICAS ANEL Z3 X Z3.\nENCONTRAR CARACTERÍSTICAS ANEL Z3 X Z3. CARACTERÍSTICAS ANEL
72,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nAFIRMAÇÃO 1 A B ELEMENTO GRUPO A B DIFERENTE ENTÃO A^2 B^2 DIFERENTE. AFIRMAÇÃO 2 CONJUNTO ROTAÇÃO GRUPO DIEDRO SUBGRUPO CÍCLICO.\n\nOpções disponíveis:\na. Verdade, Verdade\nb. Falso, Falso\nc. Verdadeiro, Falso\nd. Falso, Verdadeiro\n0. FALSO, FALSO\n\n## Step 1: AFIRMAÇÃO 1 A B ELEMENTO GRUPO DIFERENTE ENTÃO A^2 B^2 DIFERENTE VERDADEIRO.\n\n## Step"
73,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nAFIRMAÇÃO 1 GRUPO AB^-2 B^-2A^-2. AFIRMAÇÃO 2 GRUPO AB^N A^NB^N.\n\nOpções disponíveis:\na. Verdade, Verdade\nb. Falso, Falso\nc. Verdadeiro, Falso\nd. Falso, Verdadeiro\n0. FALSO, FALSO\n\n## Step 1: EXAMINAR AFIRMAÇÃO 1 AFIRMAÇÃO 1 AFIRMAÇÃO 1 AFIRMAÇÃO 1 AFIRMAÇÃO 1 AFIR"
74,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nAFIRMAÇÃO 1 S_N ABELIANO NÃO N >= 3. AFIRMAÇÃO 2 A PERMUTAÇÃO CICLO M B PERMUTAÇÃO CICLO N ENTÃO AB LCM M N.\n\nOpções disponíveis:\na. Verdade, Verdade\nb. Falso, Falso\nc. Verdadeiro, Falso\nd. Falso, Verdadeiro\n0. FALSO, FALSO\n\n## Step 1: AFIRMAÇÃO 1 S_N ABELIANO N >= 3 VERIFICAR.\nAFIRMAÇÃO 1 S_N ABELIANO N >= 3"
75,Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nENCONTRAR CARACTERÍSTICA ANEL Z X Z.\n\nOpções disponíveis:\na. 0\nb. 3\nc. 12\nd. 30\n0\n\n## Step 1: ENCONTRAR CARACTERÍSTICA ANEL Z X Z.\nENCONTRAR CARACTERÍSTICA ANEL Z X Z.\n\n## Step 2: ENCONTRAR CARACTERÍSTICA
76,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nAFIRMAÇÃO 1 N > 1 CONJUNTO 1, 2,..., N-1 GRUPO MULTIPLICAÇÃO MÓDULO N. AFIRMAÇÃO 2 INTEIRO X 63X MÓDULO 100 1.\n\nOpções disponíveis:\na. Verdade, Verdade\nb. Falso, Falso\nc. Verdadeiro, Falso\nd. Falso, Verdadeiro\n0. FALSO, FALSO\n\n## Step 1: AFIRMAÇÃO 1 N 1 CONJUNTO 1 2 ... N-1 MULTIPLICAÇÃO MÓ"
77,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nAFIRMAÇÃO 1 K SUBGRUPO ZERO DIFERENTE GRUPO ABELIANO LIVRE GERADO FINITO K ABELIANO LIVRE. AFIRMAÇÃO 2 K SUBGRUPO ZERO DIFERENTE GRUPO ABELIANO LIVRE GERADO FINITO G/K ABELIANO LIVRE.\n\nOpções disponíveis:\na. Verdade, Verdade\nb. Falso, Falso\nc. Verdadeiro, Falso\nd. Falso, Verdadeiro\n0. FALSO, FALSO\n\n## Step 1: LER AFIRMAÇÃO 1 K SUBGRUPO ZERO DIFERENTE GRUPO ABELIANO LIVRE GERADO FINITO GERAR.\nAF"
78,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nDECLARAÇÃO 1 PRODUTO DIRETO EXTERNO GRUPO CÍCLICO CÍCLICO. DECLARAÇÃO 2 PRODUTO DIRETO EXTERNO D_3 D_4 ISOMÓRFICO D_12.\n\nOpções disponíveis:\na. Verdade, Verdade\nb. Falso, Falso\nc. Verdadeiro, Falso\nd. Falso, Verdadeiro\n0. FALSO, FALSO\n\n## Step 1: ANALISAR DECLARAÇÃO 1\nDECLARAÇÃO 1 DIRETO EXTERNO PRODUTO DIRETO EXTERNO CÍCLICO.\n\n#

In [89]:
df_llama['initial'] = df_llama['resp'].progress_apply(get_initial)
df_llama['valid'] = df_llama['initial'].apply(replace_answer_data)
df_llama['is_valid'] = df_llama['is_valid'] = df_llama['valid']!= 'erro'
df_llama['valid'].value_counts()

100%|██████████| 15136/15136 [00:00<00:00, 588876.28it/s]


valid
erro    14992
0         102
3          17
1          13
2          12
Name: count, dtype: int64

In [90]:
df_llama['is_valid'].value_counts(), df_llama['is_valid'].value_counts(normalize=True) * 100

(is_valid
 False    14992
 True       144
 Name: count, dtype: int64,
 is_valid
 False    99.048626
 True      0.951374
 Name: proportion, dtype: float64)

In [91]:
df_valid = df_llama[df_llama['valid']!= 'erro']
df_valid.shape, df_valid['is_valid'].value_counts()

((144, 9),
 is_valid
 True    144
 Name: count, dtype: int64)

In [128]:
df_valid['is_valid'].value_counts(normalize=True) * 100

is_valid
True    100.0
Name: proportion, dtype: float64

# Load Deepseek


In [6]:
def extrair_primeira_palavra(texto):
    padrao = r"Responda apenas com a letra correspondente à resposta correta\.\s*(\S+)"
    match = re.search(padrao, texto)
    return match.group(1) if match else None

In [3]:
df_deep = pd.read_csv("/home/annap/Documents/chatbot-project/EVAL/DATASETS/INFERENCES/mmlu_eval_DeepSeek-R1.csv")
#pd.read_csv("/home/annap/Documents/chatbot-project/EVAL/DATASETS/MMLU/mmlu_eval_deep_seek.csv")
df_deep.shape

(15136, 5)

In [9]:
df_deep['first_word'] = df_deep['model_output_DeepSeek-R1'].progress_apply(extrair_primeira_palavra)
df_deep.loc[15:20]

100%|██████████| 15136/15136 [00:00<00:00, 614182.61it/s]


,question_id,formatted_prompt,subject,answer,model_output_DeepSeek-R1,first_word
15,16,Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nINVERSO I GRUPO MULTIPLICATIVO 1 -1 I -I.\n\nOpções disponíveis:\na. 1\nb. -1\nc. i\nd. -i\n\nResponda apenas com a letra correspondente à resposta correta.,abstract_algebra,2,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nINVERSO I GRUPO MULTIPLICATIVO 1 -1 I -I.\n\nOpções disponíveis:\na. 1\nb. -1\nc. i\nd. -i\n\nResponda apenas com a letra correspondente à resposta correta. Por favor, ignore os comentários e as expreções.\n*/\n\nOkay, so I have this question here: ""INVERSO I GRUPO MULTIPLICATIVO 1 -1 I -I."" Hmm, I'm not entirely sure",Por
16,17,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nPRODUTO CALCULAR ANEL 2 3 3 5 Z5 X Z9.\n\nOpções disponíveis:\na. (1,1)\nb. (3,1)\nc. (1,6)\nd. (3,6)\n\nResponda apenas com a letra correspondente à resposta correta.",abstract_algebra,2,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nPRODUTO CALCULAR ANEL 2 3 3 5 Z5 X Z9.\n\nOpções disponíveis:\na. (1,1)\nb. (3,1)\nc. (1,6)\nd. (3,6)\n\nResponda apenas com a letra correspondente à resposta correta. Por favor, sem mais explicações.\n*/\n\nOk, let me try to figure out this problem. The question is asking to calculate the product of two numbers, which are given in a code-like format: ANEL 2 3 3",Por
17,18,Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nCONJUNTO NÚMEROS REAIS OPERAÇÃO MULTIPLICAÇÃO GRUPO NÃO PORQUE.\n\nOpções disponíveis:\na. multiplicação não é uma operação binária\nb. multiplicação não é associativa\nc. O elemento de identidade não existe\nd. zero não tem inverso\n\nResponda apenas com a letra correspondente à resposta correta.,abstract_algebra,3,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nCONJUNTO NÚMEROS REAIS OPERAÇÃO MULTIPLICAÇÃO GRUPO NÃO PORQUE.\n\nOpções disponíveis:\na. multiplicação não é uma operação binária\nb. multiplicação não é associativa\nc. O elemento de identidade não existe\nd. zero não tem inverso\n\nResponda apenas com a letra correspondente à resposta correta. Por favor, sem mais explicações.\n*/\n\nOkay, so I have this question about real number sets, operations, multiplication, and groups. The question is asking why the set is not a group. The options are a, b, c,",Por
18,19,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nAFIRMAÇÃO 1 GRUPO ORDEM P 2 P PRIMO ABELIANO. AFIRMAÇÃO 2 P PRIMO FIXO P SUBGRUPO SYLOW GRUPO G SUBGRUPO NORMAL G ÚNICO P SUBGRUPO SYLOW G.\n\nOpções disponíveis:\na. Verdade, Verdade\nb. Falso, Falso\nc. Verdadeiro, Falso\nd. Falso, Verdadeiro\n\nResponda apenas com a letra correspondente à resposta correta.",abstract_algebra,0,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nAFIRMAÇÃO 1 GRUPO ORDEM P 2 P PRIMO ABELIANO. AFIRMAÇÃO 2 P PRIMO FIXO P SUBGRUPO SYLOW GRUPO G SUBGRUPO NORMAL G ÚNICO P SUBGRUPO SYLOW G.\n\nOpções disponíveis:\na. Verdade, Verdade\nb. Falso, Falso\nc. Verdadeiro, Falso\nd. Falso, Verdadeiro\n\nResponda apenas com a letra correspondente à resposta correta. Por favor, avalie cada afirmação separadamente.\n\nPrimeiro, vamos analisar cada afirmação individualmente.\n\nAFIRMAÇÃO 1: ""GRUP

# Join corretos entre llama e qween

In [129]:
ids_llama = df_llama[df_llama['is_valid'] == True]['question_id'].unique()
ids_qwen = df_qwen[df_qwen['is_valid'] == True]['question_id'].unique()

df_valid_llama = df_llama[ df_llama['question_id'].isin(ids_qwen) & df_llama['question_id'].isin(ids_llama)].rename(columns={'valid':'resp_modelo'})
df_valid_qwen = df_qwen[ df_qwen['question_id'].isin(ids_llama) & df_qwen['question_id'].isin(ids_qwen)].rename(columns={'valid':'resp_modelo'})

df_valid_llama = df_valid_llama[['question_id', 'formatted_prompt', 'subject', 'answer', 'resp_modelo']]
df_valid_qwen = df_valid_qwen[['question_id', 'formatted_prompt', 'subject', 'answer', 'resp_modelo']]

df_valid_llama.shape, df_valid_qwen.shape

((2914, 5), (2914, 5))

É importante apresentar essa análise estatística da distribuição dos dados antes e depois de filtrar os considerados válidos.

In [134]:
df_qwen['answer'].value_counts(normalize=True)*100

answer
3    26.605444
2    25.588002
1    24.722516
0    23.084038
Name: proportion, dtype: float64

In [135]:
df_valid_llama['answer'].value_counts(normalize=True)*100

answer
3    26.835964
2    26.321208
1    24.914207
0    21.928620
Name: proportion, dtype: float64

In [133]:
df_valid_llama['resp_modelo'].value_counts(normalize=True)*100

resp_modelo
1    38.297872
0    31.365820
3    20.178449
2    10.157859
Name: proportion, dtype: float64

# Acertos e erros

## Llama

In [139]:
df_valid_llama['valid_answers'] = df_valid_llama['answer'] == df_valid_llama['resp_modelo']
df_valid_llama['valid_answers'].value_counts(), df_valid_llama['valid_answers'].value_counts(normalize=True) * 100

(valid_answers
 False    2010
 True      904
 Name: count, dtype: int64,
 valid_answers
 False    68.977351
 True     31.022649
 Name: proportion, dtype: float64)

## Qwen

In [138]:
df_valid_qwen['valid_answers'] = df_valid_qwen['answer'] == df_valid_qwen['resp_modelo']
df_valid_qwen['valid_answers'].value_counts(), df_valid_qwen['valid_answers'].value_counts(normalize=True) * 100

(valid_answers
 False    1766
 True     1148
 Name: count, dtype: int64,
 valid_answers
 False    60.603981
 True     39.396019
 Name: proportion, dtype: float64)

# Load Phi

In [63]:
df = pd.read_csv("/home/annap/Documents/chatbot-project/EVAL/DATASETS/MMLU/mmlu_eval_phi_4.csv")
df.head()

,question_id,formatted_prompt,subject,answer,model_output
0,0,Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nGRAU ENCONTRAR EXTENSÃO CAMPO Q RAIZ QUADRADA 2 RAIZ QUADRADA 3 RAIZ QUADRADA 18 SOBRE Q.\n\nOpções disponíveis:\na. 0\nb. 4\nc. 2\nd. 6\n\nResponda apenas com a letra correspondente à resposta correta.,abstract_algebra,1,Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nGRAU ENCONTRAR EXTENSÃO CAMPO Q RAIZ QUADRADA 2 RAIZ QUADRADA 3 RAIZ QUADRADA 18 SOBRE Q.\n\nOpções disponíveis:\na. 0\nb. 4\nc. 2\nd. 6\ne. 8\n\nQual é a resposta correta?
1,1,Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nP 1 2 5 4 2 3 S5 ÍNDICE ENCONTRAR P S5.\n\nOpções disponíveis:\na. 8\nb. 2\nc. 24\nd. 120\n\nResponda apenas com a letra correspondente à resposta correta.,abstract_algebra,2,Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nP 1 2 5 4 2 3 S5 ÍNDICE ENCONTRAR P S5.\n\nOpções disponíveis:\na. 8\nb. 2\nc. 24\nd. 120\ne. 6\n\nQual é a resposta correta?
2,2,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nZERO TODOS ENCONTRAR CAMPO FINITO INDICADO POLINÔMIO COEFICIENTE X 5 MAIS 3 X 3 MAIS X 2 MAIS 2 X Z5.\n\nOpções disponíveis:\na. 0\nb. 1\nc. 0,1\nd. 0,4\n\nResponda apenas com a letra correspondente à resposta correta.",abstract_algebra,3,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nZERO TODOS ENCONTRAR CAMPO FINITO INDICADO POLINÔMIO COEFICIENTE X 5 MAIS 3 X 3 MAIS X 2 MAIS 2 X Z5.\n\nOpções disponíveis:\na. 0\nb. 1\nc. 0,1\nd. 0,4\ne. 0,2,4\n\nQual é a resposta correta? Escreva apenas a letra, sem explicação."
3,3,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nAFIRMAÇÃO 1 GRUPO FATORES GRUPO NÃO-ABELIANO NÃO-ABELIANO. AFIRMAÇÃO 2 K SUBGRUPO NORMAL H H SUBGRUPO NORMAL G K SUBGRUPO NORMAL G.\n\nOpções disponíveis:\na. Verdade, Verdade\nb. Falso, Falso\nc. Verdadeiro, Falso\nd. Falso, Verdadeiro\n\nResponda apenas com a letra correspondente à resposta correta.",abstract_algebra,1,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nAFIRMAÇÃO 1 GRUPO FATORES GRUPO NÃO-ABELIANO NÃO-ABELIANO. AFIRMAÇÃO 2 K SUBGRUPO NORMAL H H SUBGRUPO NORMAL G K SUBGRUPO NORMAL G.\n\nOpções disponíveis:\na. Verdade, Verdade\nb. Falso, Falso\nc. Verdadeiro, Falso\nd. Falso, Verdadeiro\ne. Não posso determinar\n\nResposta:"
4,4,Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nPRODUTO ENCONTRAR POLINÔMIO ANEL POLINÔMIO F X 4 X MENOS 5 G X 2 X 2 MENOS 4 X MAIS 2 Z8 X.\n\nOpções disponíveis:\na. 2x^2 + 5\nb. 6x^2 + 4x + 6\nc. 0\nd. x^2 + 1\n\nResponda apenas com a letra correspondente à resposta correta.,abstract_algebra,1,Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nPRODUTO ENCONTRAR POLINÔMIO ANEL POLINÔMIO F X 4 X MENOS 5 G X 2 X 2 MENOS 4 X MAIS 2 Z8 X.\n\nOpções disponíveis:\na. 2x^2 + 5\nb. 6x^2 + 4x + 6\nc. 0\nd. x^2 + 1\ne. 2x^2 - 2x + 2\n\nQual é a resposta correta?


# Load GLOSA

In [3]:
df_qwen = pd.read_csv("/home/annap/Documents/chatbot-project/EVAL/DATASETS/MMLU/mmlu_eval_Qwen_2_5_7B_glosa.csv")

In [ ]:
#RESPOSTA: c.

,model_output
70,Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nTEOREMA FERMAT 3^47 23 DIVIDIR RESTO ENCONTRAR.\n\nOpções disponíveis:\na. 1\nb. 2\nc. 3\nd. 4\n0
71,Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nENCONTRAR CARACTERÍSTICA ANEL Z_3 X Z_3.\n\nOpções disponíveis:\na. 0\nb. 3\nc. 12\nd. 30\n0
72,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nAFIRMAÇÃO 1 A B ELEMENTO GRUPO A B DIFERENTE ENTÃO A^2 B^2 DIFERENTE. AFIRMAÇÃO 2 CONJUNTO ROTAÇÃO GRUPO DIEDRO SUBGRUPO CÍCLICO.\n\nOpções disponíveis:\na. Verdade, Verdade\nb. Falso, Falso\nc. Verdadeiro, Falso\nd. Falso, Verdadeiro\n0"
73,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nAFIRMAÇÃO 1 GRUPO AB^-2 B^-2A^-2. AFIRMAÇÃO 2 GRUPO AB^N A^NB^N.\n\nOpções disponíveis:\na. Verdade, Verdade\nb. Falso, Falso\nc. Verdadeiro, Falso\nd. Falso, Verdadeiro\n0"
74,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nAFIRMAÇÃO 1 S_N ABELIANO NÃO N >= 3. AFIRMAÇÃO 2 A PERMUTAÇÃO CICLO M B PERMUTAÇÃO CICLO N ENTÃO AB LCM M N.\n\nOpções disponíveis:\na. Verdade, Verdade\nb. Falso, Falso\nc. Verdadeiro, Falso\nd. Falso, Verdadeiro\n0"
75,Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nENCONTRAR CARACTERÍSTICA ANEL Z X Z.\n\nOpções disponíveis:\na. 0\nb. 3\nc. 12\nd. 30\n0
76,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nAFIRMAÇÃO 1 N > 1 CONJUNTO 1, 2, ..., N-1 GRUPO MULTIPLICAÇÃO MÓDULO N. AFIRMAÇÃO 2 INTEIRO X 63X MÓDULO 100 1.\n\nOpções disponíveis:\na. Verdade, Verdade\nb. Falso, Falso\nc. Verdadeiro, Falso\nd. Falso, Verdadeiro\n0"
77,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nAFIRMAÇÃO 1 K SUBGRUPO ZERO DIFERENTE GRUPO ABELIANO LIVRE GERADO FINITO K ABELIANO LIVRE. AFIRMAÇÃO 2 K SUBGRUPO ZERO DIFERENTE GRUPO ABELIANO LIVRE GERADO FINITO G/K ABELIANO LIVRE.\n\nOpções disponíveis:\na. Verdade, Verdade\nb. Falso, Falso\nc. Verdadeiro, Falso\nd. Falso, Verdadeiro\n0"
78,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nDECLARAÇÃO 1 PRODUTO DIRETO EXTERNO GRUPO CÍCLICO CÍCLICO. DECLARAÇÃO 2 PRODUTO DIRETO EXTERNO D_3 D_4 ISOMÓRFICO D_12.\n\nOpções disponíveis:\na. Verdade, Verdade\nb. Falso, Falso\nc. Verdadeiro, Falso\nd. Falso, Verdadeiro\n0"
79,"Você é um assistente virtual altamente capacitado. Sua tarefa é ler uma pergunta e escolher a resposta correta entre as opções fornecidas.\n\nPergunta:\nAFIRMAÇÃO 1 GRUPO ABELIANO LIVRE ZERO DIFERENTE BASE INFINITO. AFIRMAÇÃO 2 GRUPO ABELIANO LIVRE POSTO 2 BASE INFINITO.\n\nOpções disponíveis:\na. Verdade, Verdade\nb. Falso, Falso\nc. Verdadeiro, Falso\nd. Falso, Verdadeiro\n0"
